# Módulos

In [14]:
import panel as pn 
import panel.widgets as pnw
pn.extension('plotly')
import pandas as pd
import numpy as np
from panel.template import DarkTheme
import plotly.graph_objects as go
import holoviews as hv
import math 
from plotly.subplots import make_subplots
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

# Clase

In [15]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Funciones para las trayectorias

In [16]:
def bm_2d(n_steps, speed, s_pos):
    velocity = Vec2d(speed, 0)

    BM_2d_df = pd.DataFrame(columns = ["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": s_pos[0], "y_pos": s_pos[1]}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{"x_pos": BM_2d_df.x_pos[i] + velocity.x, "y_pos": BM_2d_df.y_pos[i] + velocity.y}])

        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    return BM_2d_df

def crw_2d(n_steps, speed, exponent, s_pos):
    fig_BM_3d = go.Figure()

    velocity = Vec2d(exponent,0)
    BM_3d_df = pd.DataFrame(columns=['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])
    BM_3d_df=pd.concat([BM_3d_df,temp_df], ignore_index=True)

    for i in range(n_steps-1):
        turn_angle = wrapcauchy.rvs(exponent, size=1)
        velocity = velocity.rotated(turn_angle)
        temp_df = pd.DataFrame([{'x_pos':BM_3d_df.x_pos[i]+velocity.x, 'y_pos':BM_3d_df.y_pos[i]+velocity.y}])
        BM_3d_df = pd.concat([BM_3d_df,temp_df], ignore_index=True)

    return BM_3d_df

def levy(n_steps,alpha,beta,exponent,location,s_pos):
    Levy_3d_df = pd.DataFrame(columns=['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])
    Levy_3d_df=pd.concat([Levy_3d_df,temp_df], ignore_index=True)

    velocity = Vec2d(levy_stable.rvs(alpha, beta, size=1)[0],levy_stable.rvs(alpha, beta, size=1)[0])

    for i in range(n_steps-1):
        speed = Vec2d(levy_stable.rvs(alpha, beta, size=1)[0],levy_stable.rvs(alpha, beta, size=1)[0])
        velocity = velocity + speed
        velocity = velocity.rotated(wrapcauchy.rvs(c=exponent,size=1))
        temp_df = pd.DataFrame([{'x_pos':Levy_3d_df.x_pos[i]+velocity.x, 'y_pos':Levy_3d_df.y_pos[i]+velocity.y}])
        Levy_3d_df = pd.concat([Levy_3d_df,temp_df], ignore_index=True)
    return Levy_3d_df

## Funciones generales

In [17]:
def euclidean_function(p1,p2):
    distance = np.sqrt(np.square(p2.x_pos-p1.x_pos)+np.square(p2.y_pos-p1.y_pos))
    return distance

def msd(pi,pf):
    vector_result_x = pf.x_pos - pi.x_pos
    vector_result_y = pf.y_pos - pi.y_pos
    vector_square = np.square(vector_result_x) + np.square(vector_result_y)
    return vector_square

def angle_calculate(pi,pm,pf):
    #punto 0 a punto uno VECTOR UNO
    vector1_x= pm.x_pos-pi.x_pos
    vector1_y= pm.y_pos-pi.y_pos
    #punto 1 a punto dos VECTOR DOS
    vector2_x = pf.x_pos-pm.x_pos
    vector2_y = pf.y_pos-pm.y_pos
    
    escalar = (vector2_x * vector1_x) + (vector2_y*vector1_y)
    vector_pi = np.sqrt(np.square(vector1_x)+np.square(vector1_y))
    vector_pf = np.sqrt(np.square(vector2_x)+np.square(vector2_y))
    cos_teta = escalar / (vector_pi * vector_pf)
    
    direction= (vector1_x * vector2_y) - (vector2_x * vector1_y)
    
    if(direction < 0):
        teta = np.arccos(round(cos_teta,15)) * -1
    else:
        teta = np.arccos(round(cos_teta,15))
    return teta

## Funciones para las métricas

In [18]:
def path_lenght(traj):
    trajectory = np.array([euclidean_function(traj.iloc[i -1], traj.iloc[i]) for i in range(1, traj.shape[0])])
    pl_trajectory = np.cumsum(trajectory)
    return pl_trajectory

def mean_square_displacement(traj):
    MSD_TRAJ = np.empty(shape=(0), dtype=float)
    for tau in range(1,traj.shape[0]):
        sum=0
        stop = traj.shape[0] - tau
        for n in range(0,stop):
            aux = msd(traj.iloc[n], traj.iloc[n+tau])
            sum = sum + aux
        mean = sum / stop
        MSD_TRAJ = np.append(MSD_TRAJ, mean)
    return MSD_TRAJ

# Widgets

In [23]:
rd_trajectories = pnw.RadioButtonGroup(button_type='warning',name='RadioButton Trajectories',value='Default',options=['Brownian Motion','CRW','Lèvy'])
slider_steps = pnw.IntSlider(name='Steps',value=100, width=250,step=20,start=100,end=1000)
x_value = pnw.IntInput(name='X Value',value=0, step=1, start=-100, end=100)
y_value = pnw.IntInput(name='Y Value',value=0, step=1, start=-100, end=100)
slider_beta = pnw.IntSlider(name='Beta',value=1, width=250,step=1,start=1,end=10)
slider_alpha = pnw.FloatSlider(name='Alpha',value=1, width=250,step=0.1,start=0.1,end=2)
slider_speed = pnw.IntSlider(name='Speed',value=1, width=250,step=1,start=1,end=10)
slider_exponent = pnw.FloatSlider(name='Exponent',value=0.1, width=250,step=0.1,start=0.1,end=0.9)
slider_location = pnw.IntSlider(name='Location',value=1, width=250,step=1,start=1,end=10)
radio_metrics = pnw.RadioBoxGroup(name='Metrics', options=['Path Lenght','MSD'], inline=True)

## Funciones para crear figuras

In [47]:
def create_figure(data,metrics, info):
    figure = make_subplots(
        rows=1, 
        cols=2,  
          specs=[[{"type": "scene"},{"type": "xy"}]],      
    )
    
    if info == "bm":
        color = "red"
    elif info == "crw":
        color = "blue"
    elif info == "levy":
        color = "green"
        
    figure.add_trace(
        go.Scatter3d(
            x = data.x_pos,
            y = data.y_pos,
            z = data.index,
            marker = {"size": 2},
            line = {"color": color , "width": 2},
            mode = "lines",
            name = info,
            showlegend = True
        )
    )
    figure.add_trace(
        go.Scatter(
            x = np.arange(len(metrics)),
            y = metrics,
            marker = dict(size=2),
            line = dict(width=3),
            mode = 'lines',
            name = "path_length_CRW_6",
            showlegend = True
        )
    )
    figure.update_layout(
        go.Layout(
            title=go.layout.Title(text="A Figure Specified By A Graph Object"),
            width=1000,
            height=500
        )
    )
 
    return figure


In [45]:
def create_trajectory_bm(steps ,speed ,x ,y ,metrics_value):
    data = bm_2d(steps,speed, s_pos=[x, y])
    info = "bm"
    
    if(metrics_value == "MSD"):
        metrics = mean_square_displacement(data)
    elif(metrics_value == "Path Lenght"):
        metrics = path_lenght(data)
        
    figure = create_figure(data, metrics, info)
    return figure

def create_trajectory_crw(steps, speed,exponent,x,y,metrics_value):
    data=crw_2d(steps,speed,exponent, s_pos=[x, y])
    info = "crw"
    
    if(metrics_value == "MSD"):
        metrics = mean_square_displacement(data)
    elif(metrics_value == "Path Lenght"):
        metrics = path_lenght(data)
        
    figure = create_figure(data,metrics,info)
    return figure

def create_trajectory_levy(steps,alpha,beta,exponent,location,x,y,metrics_value):
    data = levy(steps,alpha,beta,exponent,location,s_pos=[x,y])
    info = "levy"
    
    if(metrics_value == "MSD"):
        metrics = mean_square_displacement(data)
    elif(metrics_value == "Path Lenght"):
        metrics = path_lenght(data)
        
    figure = create_figure(data,metrics,info)
    return figure

def components_visibility():
    if rd_trajectories.value == 'Brownian Motion':
        slider_alpha.visible = False
        slider_beta.visible = False
        slider_exponent.visible = False
        slider_location.visible = False
        
    elif rd_trajectories.value == "CRW":
        slider_alpha.visible = False
        slider_beta.visible = False
        slider_exponent.visible = True
        slider_location.visible = False
        
    elif rd_trajectories.value == "Lèvy":
        slider_alpha.visible = True
        slider_beta.visible = True
        slider_exponent.visible = True
        slider_location.visible = True

In [48]:
@pn.depends(rd_trajectories, slider_speed, slider_steps,slider_alpha,slider_beta,slider_exponent,slider_location,x_value,y_value,radio_metrics)
def validate_trajectory(rd_trajectories, slider_speed, slider_steps,slider_alpha,slider_beta,slider_exponent,slider_location,x_value,y_value,radio_metrics):
    components_visibility()
    if rd_trajectories == 'Brownian Motion':
        return create_trajectory_bm(slider_steps, slider_speed, x_value,y_value,radio_metrics)
    elif rd_trajectories == "CRW":
        return create_trajectory_crw(slider_steps, slider_speed,slider_exponent,x_value,y_value,radio_metrics)
    elif rd_trajectories == "Lèvy":
        return create_trajectory_levy(slider_steps,slider_alpha,slider_beta,slider_exponent,slider_location,x_value,y_value,radio_metrics)

layout = pn.Row(
            pn.Column(rd_trajectories,                      
                     pn.Row(slider_alpha,slider_beta),
                     pn.Row(slider_exponent,slider_location,slider_steps),
                     pn.Row(x_value,y_value),
                     validate_trajectory,
                     pn.Column(radio_metrics)                   
            )
        )
layout.servable()

Row
    [0] Column
        [0] RadioButtonGroup(button_type='warning', name='RadioButton T..., options=['Brownian Motion', ...], value='CRW')
        [1] Row
            [0] FloatSlider(end=2, name='Alpha', start=0.1, value=1, visible=False, width=250)
            [1] IntSlider(end=10, name='Beta', start=1, value=1, visible=False, width=250)
        [2] Row
            [0] FloatSlider(end=0.9, name='Exponent', start=0.1, value=0.1, width=250)
            [1] IntSlider(end=10, name='Location', start=1, value=1, visible=False, width=250)
            [2] IntSlider(end=1000, name='Steps', start=100, step=20, value=620, width=250)
        [3] Row
            [0] IntInput(end=100, name='X Value', start=-100)
            [1] IntInput(end=100, name='Y Value', start=-100)
        [4] ParamFunction(function)
        [5] Column
            [0] RadioBoxGroup(inline=True, name='Metrics', options=['Path Lenght', 'MSD'], value='MSD')

In [ ]:
template = pn.template.FastListTemplate(
    site="Panel", 
    title="Dashboard Final",
    theme="dark",
    theme_toggle=False,
    header_background="#F08080",
    sidebar=[pn.pane.Markdown("## Settings"), rd_trajectories, slider_steps,x_value,y_value,slider_speed],
    #main=[pn.pane.HoloViews(hv.DynamicMap(sine) + hv.DynamicMap(cosine), sizing_mode="stretch_both")]
).servable();